# Creativity Evaluation Dataset Preparation

Create New Model Instance For the testing

In [3]:
import numpy as np
import requests
from langchain import PromptTemplate
import requests
from langchain import PromptTemplate
from tqdm import tqdm
import psutil
from datetime import datetime
import pandas as pd, json

In [4]:
with open(
    "/home/llama/Personal_Directories/srb/causalllm-main/causal/creativity_eval/stories/ttcw_short_stories.json",
    "r",
) as f:
    stories = json.load(f)

print(f"Length of Stories {len(stories)}")
new_yorker_stories = [story for story in stories if "GPT3.5" in story["story_id"]]

print(f"Length of new_yorker Stroies {len(new_yorker_stories)}")
df = pd.DataFrame(new_yorker_stories)
df.drop(columns=["story_id"], inplace=True)
df.to_csv("/home/llama/Personal_Directories/srb/causalllm-main/resource/creativity/story_plot.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/llama/Personal_Directories/srb/causalllm-main/causal/creativity_eval/stories/ttcw_short_stories.json'

# Create Dataset For The Model

In [8]:
local_data_csv = "/Users/lujun.li/projects/causallm-github/causalllm/script/creativity/story_plot.csv"
df = pd.read_csv(local_data_csv)

In [9]:
# Two hyper parameters of max diff = 200 and max iteration time = 10
def generate_story_by_loop(
    plot,
    initial_prompt,
    recurrent_check_prompt,
    word_count_requirement,
    temperature,
    max_new_tokens,
    max_length,
    api_url=f"http://{host}:{port}/generate-text",
):
    initial_prompt_template = PromptTemplate.from_template(initial_prompt)
    recurrent_check_prompt_template = PromptTemplate.from_template(
        recurrent_check_prompt
    )
    prompt = initial_prompt_template.format(
        plot=plot, word_count=word_count_requirement
    )
    payload = {
        "text": prompt,
        "temperature": temperature,
        "max_new_tokens": max_new_tokens,
        "max_length": max_length,
    }
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        story = remove_inst_tags_llama(response.json()["result"][0]["generated_text"])
    current_word_count = len(story.split())
    repetition_times = 0
    while (word_count_requirement - current_word_count > max_diff) and repetition_times < max_iteration_times:
        prompt = recurrent_check_prompt_template.format(
            story=story,
            word_count=word_count_requirement,
            current_word_count=current_word_count,
        )
        payload["text"] = prompt
        response = requests.post(api_url, json=payload)
        if response.status_code == 200:
            story = remove_inst_tags_llama(response.json()["result"][0]["generated_text"])
            current_word_count = len(story.split())
            print (f"current length: {current_word_count}")
            repetition_times = repetition_times +1
        
    return story

In [10]:
# from utils.huggingfaceUtil import generate_story_by_loop
initial_prompt = """
<s>[INST] <<SYS>>
Write a New Yorker-style story given the plot below. Make sure it is at least {word_count} words. Directly start with the story, do not say things like 'Here's the story [...]
<</SYS>>
Current plot: {plot}
[/INST]

"""
recurrent_check_prompt = """
<s>[INST] <<SYS>>
You wrote the story I gave you below. I requested a story with {word_count} words, but the story only has {current_word_count} words.
Can you rewrite the story to make it longer, and closer to the {word_count} word target I gave you. 
Directly start with the story, do not say things like ‘Here’s the story [...]:
<</SYS>>
Current story: {story}
[/INST]

"""
csv_filename = f"/home/llama/Personal_Directories/srb/causalllm-main/resource/creativity/{module}_{quantization}_creativity_{word_count_requirement}.csv"
temperatures = np.arange(0.1, 2.0, 0.3)
pbar = tqdm(total=len(temperatures) * len(df))
times = 0
for temperature in temperatures:
    for index, row in df.iterrows():
        plot = row["plot"]
        word_count = word_count_requirement
        updated_row = row.copy()
        story = generate_story_by_loop(
            plot = plot, 
            initial_prompt = initial_prompt, 
            recurrent_check_prompt = recurrent_check_prompt, 
            word_count_requirement = word_count,
            temperature = temperature,
            max_new_tokens = 1024, 
            max_length=4096)
        updated_row["generated_response"] = story
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        updated_row["timestamp"] = timestamp
        updated_row["temperature"] = temperature
        updated_row["model"] = f"{model}_{quantization}"
        updated_dataframe = pd.DataFrame([updated_row])
        if times == 0:
            updated_dataframe.to_csv(csv_filename, index=False, mode="w", header=not index)
        else:
            updated_dataframe.to_csv(csv_filename, index=False, mode="a", header=not index)
        pbar.update(1)
        times = times + 1
        

  0%|          | 0/84 [00:00<?, ?it/s]/home/llama/Personal_Directories/srb/causalEnv/lib64/python3.9/site-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
  1%|          | 1/84 [01:14<1:42:39, 74.21s/it]

INFO:     127.0.0.1:53784 - "POST /generate-text HTTP/1.1" 200 OK


  2%|▏         | 2/84 [02:11<1:27:44, 64.20s/it]

INFO:     127.0.0.1:45244 - "POST /generate-text HTTP/1.1" 200 OK


  4%|▎         | 3/84 [03:17<1:27:45, 65.01s/it]

...

INFO:     127.0.0.1:57870 - "POST /generate-text HTTP/1.1" 200 OK


 99%|█████████▉| 83/84 [1:27:59<01:10, 70.10s/it]

INFO:     127.0.0.1:42894 - "POST /generate-text HTTP/1.1" 200 OK


100%|██████████| 84/84 [1:29:11<00:00, 70.66s/it]

INFO:     127.0.0.1:44020 - "POST /generate-text HTTP/1.1" 200 OK
